## FABlib API References Examples

- [fablib.show_config](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.show_config)
- [fablib.list_sites](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.list_sites)
- [fablib.list_hosts](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.list_hosts)
- [fablib.new_slice](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.new_slice)
- [slice.add_node](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.add_node)
- [slice.submit](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.submit)
- [slice.get_nodes](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.get_nodes)
- [slice.list_nodes](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.list_nodesß)
- [slice.show](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.show)
- [node.execute](https://fabric-fablib.readthedocs.io/en/latest/node.html#fabrictestbed_extensions.fablib.node.Node.execute)
- [slice.delete](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.delete) 

In [1]:
import datetime
import json
import asyncio

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

User: koufalex@gmail.com bastion key is valid!
Configuration is valid


Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,C:\Users\alkou\FABRIC_tokens\id_token.json
Project ID,49f65ad7-d8a2-4ab9-8ca0-ba777a2e0ea2
Bastion Host,bastion.fabric-testbed.net
Bastion Username,koufalex_0000215529
Bastion Private Key File,C:\Users\alkou/work/fabric_config/fabric_bastion_key
Slice Public Key File,C:\Users\alkou/work/fabric_config/slice_key.pub
Slice Private Key File,C:\Users\alkou/work/fabric_config/slice_key


In [3]:
%%time
slice_name = 'DYNAMOS-on-FABRIC'
image = "default_ubuntu_24"

# Please adhere to the following regex for naming: /[a-z][a-z0-9]+/
# note: see above, renamed the agent names to only have hyphens, not underscores 

node_configurations = [
    {
        "type": "control",
        "cores": 2,
        "ram": 8,
        "disk": 20,
        "site": "LOSA",
        "host": "losa-w3.fabric-testbed.net",
    },
    {
        "type": "dynamos",
        "cores": 4,
        "ram": 16,
        "disk": 20,
        "site": "LOSA",
        "host": "losa-w3.fabric-testbed.net",
    },
    {
        "type": "agent",
        "name": "server",
        "cores": 4,
        "ram": 16,
        "disk": 20,
        "site": "LOSA",
        "host": "losa-w3.fabric-testbed.net",
    },
    {
        "type": "agent",
        "name": "clientone",
        "cores": 4,
        "ram": 16,
        "disk": 20,
        "site": "LOSA",
        "host": "losa-w3.fabric-testbed.net",
    },
    {
        "type": "agent",
        "name": "clienttwo",
        "cores": 8,
        "ram": 16,
        "disk": 20,
        "site": "LOSA",
        "host": "losa-w3.fabric-testbed.net",
    },
    {
        "type": "agent",
        "name": "clientthree",
        "cores": 4,
        "ram": 16,
        "disk": 20,
        "site": "LOSA",  # TOKY seems to not be that responsive
        "host": "losa-w3.fabric-testbed.net",  # had to change this too 
    }
]

sites = list(set([configuration["site"] for configuration in node_configurations]))
agents = [configuration["name"] for configuration in node_configurations if configuration["type"] == "agent"]
thirdparties = [configuration["name"] for configuration in node_configurations if configuration["type"] == "thirdparty"]

def create_node(slice, configuration):
    if (configuration["type"] == "control"): 
        configuration["name"] = "control"

    if (configuration["type"] == "dynamos"): 
        configuration["name"] = "dynamos"
    
    return slice.add_node(name=configuration["name"], 
                          site=configuration["site"], 
                          host=configuration["host"], 
                          cores=configuration["cores"], 
                          ram=configuration["ram"], 
                          disk=configuration["disk"], 
                          validate=False,   # changed this to False because it was failing on windows  
                          raise_exception=True, 
                          image=image)
    

CPU times: total: 0 ns
Wall time: 0 ns


In [4]:
%%time 

# Create a slice
slice = fablib.new_slice(name=slice_name)

# Add Nodes with the specific variables
# Also validate the node can be created and raise an exception in case of failure
print('Adding nodes...')
nodes = [create_node(slice, configuration) for configuration in node_configurations]
nodes_per_site = [
    (site, [node for node in nodes if node.get_site() == site])
    for site in sites
]

print('Adding network interfaces...')
interfaces_per_site = [
    (site, [node.add_component(model='NIC_Basic', name='NIC').get_interfaces()[0] for node in nodes])
    for (site, nodes) in nodes_per_site
]

print('Adding network...')
networks = [
    slice.add_l3network(name=f'Network-{site}', interfaces=interfaces, type="IPv4")
    for (site, interfaces) in interfaces_per_site
]

print(networks, [n.get_gateway() for n in networks], [n.get_subnet() for n in networks])

# Calculate the lease end time for 2 weeks from now with timezone information
lease_end_time = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(weeks=2)

# Submit the slice, using an end date 2 weeks from now (the current maximum lease time) 
# to make sure that the slice can be used for a longer period of time. Progress shows an indicator of the current progression.
# Wait until the state is finished and use an interval (it may take some time before the slice and nodes are created)
print('Creating slice...')
slice.submit(wait=True, wait_timeout=3600, wait_interval=20, progress=True, wait_jupyter='text', lease_end_time=lease_end_time);


Retry: 7, Time: 164 sec


ID,b7d18702-67d0-4749-a063-d5f97a0517fd
Name,DYNAMOS-on-FABRIC
Lease Expiration (UTC),2025-10-21 09:03:26 +0000
Lease Start (UTC),2025-10-07 09:03:27 +0000
Project ID,49f65ad7-d8a2-4ab9-8ca0-ba777a2e0ea2
State,Configuring
Email,koufalex@gmail.com
UserId,9e537cc1-dc5e-43e1-b4b2-eca83032df0c


ID,Name,Site,Type,State,Error
ce71aaaa-4f20-4922-867b-c9c255998d02,clientone,LOSA,node,Active,
693a9757-9123-4aeb-a1ec-13b709b0f42f,clientthree,LOSA,node,Active,
1178368f-df11-4871-9089-42187c92701b,clienttwo,LOSA,node,Active,
c681a6fa-738f-4ee4-b2b9-412b23dc4614,control,LOSA,node,Active,
1982b576-bb59-4c3f-b99b-143062f14ac1,dynamos,LOSA,node,Active,
fcbbbce0-ab4d-4dfe-a620-3473f37095b5,server,LOSA,node,Active,
37f43c16-9977-44ac-99d4-c123ddbe7447,Network-LOSA,LOSA,network,Ticketed,


CPU times: total: 734 ms
Wall time: 3min 6s


Exception: Failed to get slice topology: Status.FAILURE, Error [Unable to read graph C:\Users\alkou\AppData\Local\Temp\tmpzziwis1a-graphml] importing graph

In [4]:
%%time
slice = fablib.get_slice(name="DYNAMOS-on-FABRIC");
nodes = slice.get_nodes();

nodes_and_network_per_site = [
    (site, [node for node in nodes if node.get_site() == site], slice.get_network(name=f"Network-{site}"))
    for site in sites
]
networks = [network for (_, _, network) in nodes_and_network_per_site]

nodes_network_ips_per_site = [
    (site, nodes, network, network.get_available_ips(len(nodes)))
    for (site, nodes, network) in nodes_and_network_per_site
]

In [5]:
%%time
def assign_ip(site, network, available_ips, node):
    interface = node.get_interface(network_name=f"Network-{site}")
    address = available_ips.pop(0)
    network_gateway = network.get_gateway()
    network_subnet = network.get_subnet()

    network.allocate_ip(address)
    interface.ip_addr_add(addr=address, subnet=network_subnet)
    node.ip_route_add(subnet=network_subnet, gateway=network_gateway)

    # For the multisite IPv4 connection
    for network in networks:
        node.ip_route_add(subnet=network.get_subnet(), gateway=network_gateway)

    return address

ips = [assign_ip(site, network, ips, node) for (site, nodes, network, ips) in nodes_network_ips_per_site for node in nodes];

In [6]:
%%time
slice = fablib.get_slice(name="DYNAMOS-on-FABRIC");
nodes = slice.get_nodes();

for node in nodes:
    ssh_command = node.get_ssh_command().replace(
        "-i /home/fabric/work/fabric_config/slice_key", "-i ~/.ssh/keys/FABRIC-slice_key"
    ).replace(
        "-F /home/fabric/work/fabric_config/ssh_config ", ""
    )
    
    print(ssh_command);

ssh -i ~/.ssh/keys/FABRIC-slice_key ubuntu@2001:400:a100:3070:f816:3eff:fe84:397d
ssh -i ~/.ssh/keys/FABRIC-slice_key ubuntu@2001:400:a100:3070:f816:3eff:fe1b:63f7
ssh -i ~/.ssh/keys/FABRIC-slice_key ubuntu@2001:400:a100:3070:f816:3eff:fe08:6a6e
ssh -i ~/.ssh/keys/FABRIC-slice_key ubuntu@2001:400:a100:3070:f816:3eff:fe91:6d37
ssh -i ~/.ssh/keys/FABRIC-slice_key ubuntu@2001:400:a100:3070:f816:3eff:fe95:daf0
ssh -i ~/.ssh/keys/FABRIC-slice_key ubuntu@2001:400:a100:3070:f816:3eff:fead:1f7e


In [7]:
%%time
print("Uploading the node setup...")
threads = [node.upload_file_thread(local_file_path="node_scripts/node_setup.sh", remote_file_path="setup.sh")
           for node in nodes]
[thread.result() for thread in threads]


Uploading the node setup...


[<SFTPAttributes: [ size=1222 uid=1000 gid=1000 mode=0o100664 atime=1758572167 mtime=1758572168 ]>,
 <SFTPAttributes: [ size=1222 uid=1000 gid=1000 mode=0o100664 atime=1758572165 mtime=1758572165 ]>,
 <SFTPAttributes: [ size=1222 uid=1000 gid=1000 mode=0o100664 atime=1758572165 mtime=1758572165 ]>,
 <SFTPAttributes: [ size=1222 uid=1000 gid=1000 mode=0o100664 atime=1758572165 mtime=1758572165 ]>,
 <SFTPAttributes: [ size=1222 uid=1000 gid=1000 mode=0o100664 atime=1758572165 mtime=1758572165 ]>,
 <SFTPAttributes: [ size=1222 uid=1000 gid=1000 mode=0o100664 atime=1758572167 mtime=1758572168 ]>]

In [8]:
%%time
print("Executing the node setup...")
threads = [node.execute_thread(f"chmod +x setup.sh && ./setup.sh")
           for node in nodes]
[thread.result() for thread in threads]

Executing the node setup...


[("Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease\nGet:2 http://security.ubuntu.com/ubuntu noble-security InRelease [126 kB]\nGet:3 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]\nGet:4 http://security.ubuntu.com/ubuntu noble-security/main amd64 Packages [1143 kB]\nGet:5 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]\nGet:6 http://nova.clouds.archive.ubuntu.com/ubuntu noble/universe amd64 Packages [15.0 MB]\nGet:7 http://security.ubuntu.com/ubuntu noble-security/main Translation-en [195 kB]\nGet:8 http://security.ubuntu.com/ubuntu noble-security/main amd64 Components [21.6 kB]\nGet:9 http://security.ubuntu.com/ubuntu noble-security/main amd64 c-n-f Metadata [8744 B]\nGet:10 http://security.ubuntu.com/ubuntu noble-security/universe amd64 Packages [880 kB]\nGet:11 http://security.ubuntu.com/ubuntu noble-security/universe Translation-en [195 kB]\nGet:12 http://security.ubuntu.com/ubuntu noble-security/univ

In [9]:
%%time
def get_ip(node):
    interface = node.get_interface(network_name=f"Network-{node.get_site()}")
    return interface.get_ip_addr()

nodes_dict= dict()

for node in nodes[:]:
    ip = get_ip(node)
    name = node.get_name()
    nodes_dict[name] = {"ip": ip, "node": node}
    print(f"{name}: {ip}")

print(nodes_dict)


control: 10.137.3.2
dynamos: 10.137.3.3
server: 10.137.3.4
clientone: 10.137.3.5
clienttwo: 10.137.3.6
clientthree: 10.137.3.7
{'control': {'ip': '10.137.3.2', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7e3bb828a5d0>}, 'dynamos': {'ip': '10.137.3.3', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7e3ba030a490>}, 'server': {'ip': '10.137.3.4', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7e3b827a6050>}, 'clientone': {'ip': '10.137.3.5', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7e3bb8240b90>}, 'clienttwo': {'ip': '10.137.3.6', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7e3bb82ac290>}, 'clientthree': {'ip': '10.137.3.7', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7e3bbadf7ed0>}}


In [10]:
%%time
nodes_dict["control"]["node"].upload_file(local_file_path="node_scripts/install_k9s.sh", remote_file_path="k9s.sh")
nodes_dict["control"]["node"].execute(f"chmod +x k9s.sh && ./k9s.sh");

--2025-09-22 20:21:51--  https://github.com/derailed/k9s/releases/download/v0.32.5/k9s_linux_amd64.deb
Resolving github.com (github.com)... 2600:2701:5000:5001::8c52:7104, 140.82.113.4
Connecting to github.com (github.com)|2600:2701:5000:5001::8c52:7104|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/167596393/7cc41638-6a22-4598-9b02-646efaaa1053?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-09-22T21%3A11%3A32Z&rscd=attachment%3B+filename%3Dk9s_linux_amd64.deb&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-09-22T20%3A10%3A50Z&ske=2025-09-22T21%3A11%3A32Z&sks=b&skv=2018-11-09&sig=5rUv6JZ%2B2K1p4fSE8BmqoT2jyoDmTTPq1tzHWZnTwPo%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1ODU3MjgxMiwibmJmIjoxNzU4

In [11]:
# the order of the nodes is not the same as in the configuration 
# ips
# names
# names_and_ips = {n:i for n,i in names_and_ips}
# print(names_and_ips)

In [12]:
%%time
inventory = (
    f"[kube_control_plane]\n"
    f"control ansible_host={nodes_dict['control']['ip']} ip={nodes_dict['control']['ip']} etcd_member_name=etcd1\n"
    f"\n"
    f"[etcd:children]\n"
    f"kube_control_plane\n"
    f"\n"
    f"[kube_node]\n"
    f"dynamos ansible_host={nodes_dict['dynamos']['ip']} ip={nodes_dict['dynamos']['ip']}\n"
)

for node_name in nodes_dict.keys():
    if node_name not in ["dynamos", "control"]:
        inventory += f"{node_name} ansible_host={nodes_dict[node_name]['ip']} ip={nodes_dict[node_name]['ip']}\n"

with open('kubespray/inventory.ini', 'w') as f:
    f.write(inventory)

In [13]:
print(inventory)

[kube_control_plane]
control ansible_host=10.137.3.2 ip=10.137.3.2 etcd_member_name=etcd1

[etcd:children]
kube_control_plane

[kube_node]
dynamos ansible_host=10.137.3.3 ip=10.137.3.3
server ansible_host=10.137.3.4 ip=10.137.3.4
clientone ansible_host=10.137.3.5 ip=10.137.3.5
clienttwo ansible_host=10.137.3.6 ip=10.137.3.6
clientthree ansible_host=10.137.3.7 ip=10.137.3.7



In [14]:
%%time
nodes_dict['control']['node'].upload_file(local_file_path="node_scripts/control_kubespray_setup.sh", remote_file_path="kubespray_setup.sh");
nodes_dict['control']['node'].execute("chmod +x kubespray_setup.sh && ./kubespray_setup.sh");

nodes_dict['control']['node'].upload_file(local_file_path="kubespray/inventory.ini", remote_file_path="kubespray/inventory/dynamos/inventory.ini");
nodes_dict['control']['node'].upload_file(local_file_path="kubespray/ansible.cfg", remote_file_path="kubespray/ansible.cfg");
nodes_dict['control']['node'].upload_file(local_file_path="node_scripts/dot_kube.sh", remote_file_path="dot_kube.sh");
nodes_dict['control']['node'].execute("chmod +x ./dot_kube.sh");
nodes_dict['control']['node'].upload_file(local_file_path="/home/fabric/work/fabric_config/slice_key", remote_file_path="/home/ubuntu/.ssh/slice_key");
nodes_dict['control']['node'].execute("chmod 600 /home/ubuntu/.ssh/slice_key");

Cloning into 'kubespray'...
branch 'release-2.27' set up to track 'origin/release-2.27'.
Switched to a new branch 'release-2.27'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 118.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 131.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.6/219.6 kB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.5/767.5 kB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.1/118.1 kB 39.3 MB/s eta 0:00:00
ansible [core 2.16.14]
  config file = /home/ubuntu/kubespray/ansible.cfg
  configured module search path = ['/home/ubuntu/kubespray/library']
  ansible pyt

In [15]:
%%time
nodes_dict['control']['node'].upload_file(local_file_path="node_scripts/start_kubespray.sh", remote_file_path="start.sh");
nodes_dict['control']['node'].execute(f"chmod +x start.sh && ./start.sh");

[WARNING]: While constructing a mapping from
/home/ubuntu/kubespray/roles/bootstrap-os/tasks/main.yml, line 29, column 7,
found a duplicate dict key (paths). Using last defined value only.
[WARNING]: Skipping callback plugin 'ara_default', unable to load
Using /home/ubuntu/kubespray/ansible.cfg as config file

PLAY [Check Ansible version] ***************************************************
Monday 22 September 2025  20:25:46 +0000 (0:00:00.008)       0:00:00.008 ****** 

TASK [Check 2.16.4 <= Ansible version < 2.17.0] ********************************
ok: [dynamos] => {
    "changed": false,
    "msg": "All assertions passed"
}
Monday 22 September 2025  20:25:46 +0000 (0:00:00.021)       0:00:00.030 ****** 

TASK [Check that python netaddr is installed] **********************************
ok: [dynamos] => {
    "changed": false,
    "msg": "All assertions passed"
}
Monday 22 September 2025  20:25:46 +0000 (0:00:00.097)       0:00:00.127 ****** 

TASK [Check that jinja is not too old (inst

In [16]:
# This is for resetting the kubespray cluster. 
# Use this if you are troubleshooting your Kubernetes cluster
# and you want to redeploy fresh.

# nodes_dict['control']['node'].upload_file(local_file_path="node_scripts/reset_kubespray.sh", remote_file_path="reset.sh");
# nodes_dict['control']['node'].execute(f"chmod +x reset.sh && ./reset.sh");

In [17]:
# Add the relevant etcd data to the dynamos node
# nodes_dict['dynamos']['node'].upload_file(local_file_path="node_scripts/define_etcd_data.sh", remote_file_path="define_etcd_data.sh");
# nodes_dict['dynamos']['node'].execute(f"chmod +x define_etcd_data.sh && ./define_etcd_data.sh");

In [18]:
# Add the relevant etcd data to the dynamos node

# help(nodes_dict['dynamos']['node'])
# upload etcd files from filesystem instead of reading them from github
nodes_dict['dynamos']['node'].upload_directory(local_directory_path="../configuration/etcd_launch_files", remote_directory_path="./")
nodes_dict['dynamos']['node'].execute("ls etcd_launch_files")

nodes_dict['dynamos']['node'].upload_file(local_file_path="node_scripts/define_etcd_data_local.sh", remote_file_path="define_etcd_data_local.sh");
nodes_dict['dynamos']['node'].execute(f"chmod +x define_etcd_data_local.sh && ./define_etcd_data_local.sh");

agreements.json
archetype.json
datasets.json
microservices.json
optional_microservices.json
requestType.json
Create etcd-data folder
Copy config files into the folder
changing mode
Done!


In [22]:
# Preconfigure Helm for DYNAMOS and clone DYNAMOS repo
nodes_dict['control']['node'].upload_file(local_file_path="node_scripts/install_dynamos.sh", remote_file_path="dynamos.sh");
nodes_dict['control']['node'].execute(f"chmod +x dynamos.sh && ./dynamos.sh")


Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease
Hit:2 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease
Hit:3 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease
Hit:5 http://security.ubuntu.com/ubuntu noble-security InRelease
Hit:4 https://prod-cdn.packages.k8s.io/repositories/isv:/kubernetes:/core:/stable:/v1.30/deb  InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
curl is already the newest version (8.5.0-2ubuntu10.6).
gpg is already the newest version (2.4.4-2ubuntu17.3).
apt-transport-https is already the newest version (2.8.3).
The following packages were automatically installed and are no longer required:
  bridge-utils dns-root-data dnsmasq-base pigz ubuntu-fan
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.
deb [signed-by=/usr/share/keyrings/helm.gpg] https://packages.buildkite.com/helm-linux/helm-

('Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease\nHit:2 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease\nHit:3 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease\nHit:5 http://security.ubuntu.com/ubuntu noble-security InRelease\nHit:4 https://prod-cdn.packages.k8s.io/repositories/isv:/kubernetes:/core:/stable:/v1.30/deb  InRelease\nReading package lists...\nReading package lists...\nBuilding dependency tree...\nReading state information...\ncurl is already the newest version (8.5.0-2ubuntu10.6).\ngpg is already the newest version (2.4.4-2ubuntu17.3).\napt-transport-https is already the newest version (2.8.3).\nThe following packages were automatically installed and are no longer required:\n  bridge-utils dns-root-data dnsmasq-base pigz ubuntu-fan\nUse \'sudo apt autoremove\' to remove them.\n0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.\ndeb [signed-by=/usr/share/keyrings/helm.gpg] https://packages.buildkite.

In [23]:

# Configure DYNAMOS for the FABRIC nodes
agents_string = ",".join(agents)
thirdparties_string = ",".join(thirdparties)


nodes_dict['control']['node'].upload_file(local_file_path="node_scripts/configure_dynamos.sh", remote_file_path="configure_dynamos.sh");
nodes_dict['control']['node'].execute(f"chmod +x configure_dynamos.sh && ./configure_dynamos.sh {agents_string} {thirdparties_string}");

Adding agents...
- agent 'server'
- agent 'clientone'
- agent 'clienttwo'
- agent 'clientthree'

Adding third parties...


In [24]:
# Optionally override the installation scripts 
nodes_dict['control']['node'].upload_file(local_file_path="../configuration/dynamos-configuration.sh", remote_file_path="/home/ubuntu/DYNAMOS/configuration/dynamos-configuration.sh")
nodes_dict['control']['node'].upload_file(local_file_path="../configuration/fill-rabbit-pvc.sh", remote_file_path="/home/ubuntu/DYNAMOS/configuration/fill-rabbit-pvc.sh");


In [29]:
# install DYNAMOS
nodes_dict['control']['node'].execute(f"~/DYNAMOS/configuration/dynamos-configuration.sh")

DYNAMOS configuration v0.1.2
Setting up paths...
definitions_example.json copied over definitions.json to ensure a clean file
Generating RabbitMQ password...
Replacing tokens...
Installing namespaces...
Release "namespaces" has been upgraded. Happy Helming!
NAME: namespaces
LAST DEPLOYED: Mon Sep 22 21:30:28 2025
NAMESPACE: default
STATUS: deployed
REVISION: 2
TEST SUITE: None
Preparing PVC
fill-pvc v0.1.2
pod/temp-pod created
pod/temp-pod-orch created
Waiting for temp-pod to be Running...
pod/temp-pod condition met
pod/temp-pod-orch condition met
pod "temp-pod" deleted
pod "temp-pod-orch" deleted
Installing Prometheus...
"prometheus-community" already exists with the same configuration, skipping
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "prometheus-community" chart repository
Update Complete. ⎈Happy Helming!⎈
Release "prometheus" does not exist. Installing it now.
NAME: prometheus
LAST DEPLOYED: Mon Sep 22 21:31:10 2025


('DYNAMOS configuration v0.1.2\nSetting up paths...\ndefinitions_example.json copied over definitions.json to ensure a clean file\nGenerating RabbitMQ password...\nReplacing tokens...\nInstalling namespaces...\nRelease "namespaces" has been upgraded. Happy Helming!\nNAME: namespaces\nLAST DEPLOYED: Mon Sep 22 21:30:28 2025\nNAMESPACE: default\nSTATUS: deployed\nREVISION: 2\nTEST SUITE: None\nPreparing PVC\nfill-pvc v0.1.2\npod/temp-pod created\npod/temp-pod-orch created\nWaiting for temp-pod to be Running...\npod/temp-pod condition met\npod/temp-pod-orch condition met\npod "temp-pod" deleted\npod "temp-pod-orch" deleted\nInstalling Prometheus...\n"prometheus-community" already exists with the same configuration, skipping\nHang tight while we grab the latest from your chart repositories...\n...Successfully got an update from the "prometheus-community" chart repository\nUpdate Complete. ⎈Happy Helming!⎈\nRelease "prometheus" does not exist. Installing it now.\nNAME: prometheus\nLAST DEPL

In [28]:
# Optional to clean up: uninstall DYNAMOS

command = "helm uninstall agents api-gateway core orchestrator namespaces prometheus thirdparties"
# nodes_dict['control']['node'].execute(command)

Error: uninstall: Release not loaded: agents: release: not found


('', 'Error: uninstall: Release not loaded: agents: release: not found\n')

In [ ]:
# Optional delete etcd PVCs 
# nodes_dict['control']['node'].execute("kubectl get pvc --all-namespaces")

# nodes_dict['control']['node'].execute("kubectl delete pvc etcd-data-etcd-0 -n core")
# nodes_dict['control']['node'].execute("kubectl delete pvc etcd-data-etcd-1 -n core")
# nodes_dict['control']['node'].execute("kubectl delete pvc etcd-data-etcd-2 -n core")


# nodes_dict['control']['node'].execute("kubectl get pvc --all-namespaces")